**Player pair set and validation**

Objective is to get dates and player pair set for OF of ESPN Top 300. This can be used for example data and for validation.

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

from termcolor import colored


In [3]:
# Web/database stuff
# import urllib.request
# import xml.etree.ElementTree as ET
# from bs4 import BeautifulSoup
import sqlalchemy
import sqlalchemy_utils
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2

# Multiprocessing/threading
import multiprocess
import threading   # included in base

In [4]:
# Stats
from scipy.stats import mannwhitneyu
from scipy.stats import ttest_ind

In [5]:
# ML
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [115]:
# Code formatting with Jupyter black
%load_ext nb_black

<IPython.core.display.Javascript object>

# Considerations

- average players play less so harder to get common datesm

# Evaluate ESPN Top 300 and statcast data for potential matchupsm

In [8]:
! ls insight_baseball_notes/

BL_baseball_notes _google_slides.pptx
BL_baseball_notes.pptx
Ben_Lacar_Week2.pptx
Ben_Lacar_Week2_from_google.pptx
Fantrax-Team-Roster-The Jungle.csv
Full_Report_of_the_Committee_Studying_Home_Run_Rates_in_Major_League_Baseball_052418.pdf
Notes_other_links.txt
Qs_for_Eric.txt
model_building.xlsx
war_ranking_2019.csv


In [9]:
df_war_300 = pd.read_csv('insight_baseball_notes/war_ranking_2019.csv', index_col=0)
df_war_300.head()

,rank,player,team,elig_pos,pos_rank,age
0,1,Mike Trout,LAA,OF,OF1,28
1,2,Ronald Acuna Jr.,ATL,OF,OF2,21
2,3,Christian Yelich,MIL,OF,OF3,27
3,4,Mookie Betts,BOS,OF,OF4,26
4,5,Jacob deGrom,NYM,SP,SP1,31


In [184]:
df_of = df_war_300[df_war_300["pos_rank"].str.contains("OF")].copy()
df_of

,rank,player,team,elig_pos,pos_rank,age
0,1,Mike Trout,LAA,OF,OF1,28
1,2,Ronald Acuna Jr.,ATL,OF,OF2,21
2,3,Christian Yelich,MIL,OF,OF3,27
3,4,Mookie Betts,BOS,OF,OF4,26
11,12,Juan Soto,WSH,OF,OF6,20
...,...,...,...,...,...,...
277,278,Avisail Garcia,TB,OF,OF87,28
281,282,Jesse Winker,CIN,OF,OF90,26
296,297,Anthony Santander,BAL,OF,OF92,24
297,298,Mike Yastrzemski,SF,OF,OF93,29


<IPython.core.display.Javascript object>

In [185]:
df_of.head(40).tail(5)

,rank,player,team,elig_pos,pos_rank,age
138,139,Hunter Renfroe,SD,OF,OF41,27
147,148,Bryan Reynolds,PIT,OF,OF42,24
150,151,Danny Santana,TEX,OF/1B/2B,OF43,28
156,157,Kyle Schwarber,CHC,OF,OF45,26
158,159,Mitch Haniger,SEA,OF,OF46,28


<IPython.core.display.Javascript object>

In [169]:
index = 3
index_list = []
for i in range(12):
    index_list.append(index)
    index += 25
    
df_war_300.loc[index_list, :]

<IPython.core.display.Javascript object>

,rank,player,team,elig_pos,pos_rank,age
3,4,Mookie Betts,BOS,OF,OF4,26
28,29,Paul Goldschmidt,STL,1B,1B4,31
53,54,Mike Clevinger,CLE,SP,SP13,28
78,79,Yuli Gurriel,HOU,3B/1B,3B10,35
103,104,Elvis Andrus,TEX,SS,SS16,31
128,129,Corey Kluber,CLE,SP,SP32,33
153,154,Luke Voit,NYY,1B,1B22,28
178,179,Ramon Laureano,OAK,OF,OF54,25
203,204,Renato Nunez,BAL,3B/1B,3B25,25
228,229,Ken Giles,TOR,RP,RP25,28


<IPython.core.display.Javascript object>

In [174]:
index = 32
index_list = []
for i in range(12):
    index_list.append(index)
    index += 25

df_war_300.loc[index_list, :]

,rank,player,team,elig_pos,pos_rank,age
32,33.0,Marcell Ozuna,STL,OF,OF14,28.0
57,58.0,Bo Bichette,TOR,SS,SS10,21.0
82,83.0,Nicholas Castellanos,CHC,OF,OF25,27.0
107,108.0,Carlos Martinez,STL,SP/RP,SP27,27.0
132,133.0,Franmil Reyes,CLE,OF,OF39,24.0
157,158.0,Zack Wheeler,NYM,SP,SP40,29.0
182,183.0,Sean Doolittle,WSH,RP,RP20,32.0
207,208.0,Marco Gonzales,SEA,SP,SP52,27.0
232,233.0,Freddy Galvis,CIN,SS/2B,SS27,29.0
257,258.0,Starlin Castro,MIA,2B/3B,2B30,29.0


<IPython.core.display.Javascript object>

In [173]:
index = 3
index_list = []
for i in range(15):
    index_list.append(index)
    index += 20

df_war_300.loc[index_list, :]

,rank,player,team,elig_pos,pos_rank,age
3,4,Mookie Betts,BOS,OF,OF4,26
23,24,Charlie Blackmon,COL,OF,OF10,33
43,44,Noah Syndergaard,NYM,SP,SP10,26
63,64,Brad Hand,CLE,RP,RP6,29
83,84,Madison Bumgarner,SF,SP,SP20,30
103,104,Elvis Andrus,TEX,SS,SS16,31
123,124,Archie Bradley,ARI,RP,RP14,27
143,144,Dansby Swanson,ATL,SS,SS22,25
163,164,Reynaldo Lopez,CWS,SP,SP43,25
183,184,Willie Calhoun,TEX,OF,OF56,24


<IPython.core.display.Javascript object>

In [ ]:
# Get a middle tier

In [13]:
df_of.tail()

,rank,player,team,elig_pos,pos_rank,age
281,282,Jesse Winker,CIN,OF,OF90,26
285,286,Ian Desmond,COL,1B/OF,1B38,33
296,297,Anthony Santander,BAL,OF,OF92,24
297,298,Mike Yastrzemski,SF,OF,OF93,29
299,300,Leury Garcia,CWS,OF/SS,OF94,28


In [15]:
df_of_set = pd.concat([df_of.head(), df_of.tail()], axis=0)
df_of_set

,rank,player,team,elig_pos,pos_rank,age
0,1,Mike Trout,LAA,OF,OF1,28
1,2,Ronald Acuna Jr.,ATL,OF,OF2,21
2,3,Christian Yelich,MIL,OF,OF3,27
3,4,Mookie Betts,BOS,OF,OF4,26
5,6,Cody Bellinger,LAD,1B/OF,1B1,24
281,282,Jesse Winker,CIN,OF,OF90,26
285,286,Ian Desmond,COL,1B/OF,1B38,33
296,297,Anthony Santander,BAL,OF,OF92,24
297,298,Mike Yastrzemski,SF,OF,OF93,29
299,300,Leury Garcia,CWS,OF/SS,OF94,28


# Import data via database

In [17]:
# Define a database name
# Set your postgres username
dbname = "baseball"
username = "lacar"  # change this to your username

## 'engine' is a connection to a database
## Here, we're using postgres, but sqlalchemy can connect to other things too.
engine = create_engine("postgres://%s@localhost/%s" % (username, dbname))
print(engine.url)

# Working with PostgreSQL in Python
# Connect to make queries using psycopg2
con = None
con = psycopg2.connect(database=dbname, user=username)

postgres://lacar@localhost/baseball


In [18]:
# Batting stats query

sql_query = """
SELECT * FROM batting_stats18_wpid;
"""

df_batting_stats_wpid = pd.read_sql_query(sql_query, con)
df_batting_stats_wpid.head()

,index,index_x,Season,Name,Team,Age,G,AB,PA,H,...,first_name,index_y,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
0,0,63,2018.0,Mookie Betts,Red Sox,25.0,136.0,520.0,614.0,180.0,...,mookie,166,betts,mookie,605141,bettm001,bettsmo01,13611,2014.0,2019.0
1,1,67,2018.0,Mike Trout,Angels,26.0,140.0,471.0,608.0,147.0,...,mike,1789,trout,mike,545361,troum001,troutmi01,10155,2011.0,2019.0
2,2,153,2018.0,Jose Ramirez,Indians,25.0,157.0,578.0,698.0,156.0,...,jose,1417,ramirez,jose,542432,ramij004,ramirjo02,10171,2014.0,2018.0
3,68,38,2018.0,Ryder Jones,Giants,24.0,5.0,8.0,8.0,3.0,...,ryder,903,jones,ryder,624507,joner005,jonesry01,15982,2017.0,2018.0
4,3,153,2018.0,Jose Ramirez,Indians,25.0,157.0,578.0,698.0,156.0,...,jose,1418,ramirez,jose,608070,ramij003,ramirjo01,13510,2013.0,2019.0


In [25]:
df_batting_stats_wpid[df_batting_stats_wpid['name_last'].str.contains('acuna')]

,index,index_x,Season,Name,Team,Age,G,AB,PA,H,...,first_name,index_y,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
60,46,163,2018.0,Ronald Acuna Jr.,Braves,20.0,111.0,433.0,487.0,127.0,...,ronald,3,acuna,ronald,660670,acunr001,acunaro01,18401,2018.0,2019.0


## Synthesize df of pairs

In [37]:
df_of_set['name_first_lookup'] = df_of_set['player'].str.split(expand=True).loc[:, 0].str.lower()
df_of_set['name_last_lookup'] = df_of_set['player'].str.split(expand=True).loc[:, 1].str.lower()
df_of_set.head()

,rank,player,team,elig_pos,pos_rank,age,name_last_lookup,name_first_lookup
0,1,Mike Trout,LAA,OF,OF1,28,trout,mike
1,2,Ronald Acuna Jr.,ATL,OF,OF2,21,acuna,ronald
2,3,Christian Yelich,MIL,OF,OF3,27,yelich,christian
3,4,Mookie Betts,BOS,OF,OF4,26,betts,mookie
5,6,Cody Bellinger,LAD,1B/OF,1B1,24,bellinger,cody


In [41]:
df_of_set_wpid_draft1 = pd.merge(df_of_set, df_batting_stats_wpid[['name_last', 'key_mlbam', 'Team', 'WAR']], how='inner', left_on='name_last_lookup', right_on='name_last')
df_of_set_wpid_draft1.head()

,rank,player,team,elig_pos,pos_rank,age,name_last_lookup,name_first_lookup,name_last,key_mlbam,Team,WAR
0,1,Mike Trout,LAA,OF,OF1,28,trout,mike,trout,545361,Angels,9.8
1,2,Ronald Acuna Jr.,ATL,OF,OF2,21,acuna,ronald,acuna,660670,Braves,3.7
2,3,Christian Yelich,MIL,OF,OF3,27,yelich,christian,yelich,592885,Brewers,7.6
3,4,Mookie Betts,BOS,OF,OF4,26,betts,mookie,betts,605141,Red Sox,10.4
4,6,Cody Bellinger,LAD,1B/OF,1B1,24,bellinger,cody,bellinger,641355,Dodgers,3.6


In [44]:
df_of_set_wpid_draft1.iloc[0:7, :]

,rank,player,team,elig_pos,pos_rank,age,name_last_lookup,name_first_lookup,name_last,key_mlbam,Team,WAR
0,1,Mike Trout,LAA,OF,OF1,28,trout,mike,trout,545361,Angels,9.8
1,2,Ronald Acuna Jr.,ATL,OF,OF2,21,acuna,ronald,acuna,660670,Braves,3.7
2,3,Christian Yelich,MIL,OF,OF3,27,yelich,christian,yelich,592885,Brewers,7.6
3,4,Mookie Betts,BOS,OF,OF4,26,betts,mookie,betts,605141,Red Sox,10.4
4,6,Cody Bellinger,LAD,1B/OF,1B1,24,bellinger,cody,bellinger,641355,Dodgers,3.6
5,282,Jesse Winker,CIN,OF,OF90,26,winker,jesse,winker,608385,Reds,0.9
6,286,Ian Desmond,COL,1B/OF,1B38,33,desmond,ian,desmond,435622,Rockies,-0.6
7,297,Anthony Santander,BAL,OF,OF92,24,santander,anthony,santander,623993,Orioles,-0.4
8,300,Leury Garcia,CWS,OF/SS,OF94,28,garcia,leury,garcia,605244,Giants,0.4
9,300,Leury Garcia,CWS,OF/SS,OF94,28,garcia,leury,garcia,594824,Cardinals,0.4


In [60]:
# Ignore more of the pairs where both players are good and account for Garcia
df_of_set_wpid_draft2 = pd.concat([df_of_set_wpid_draft1.iloc[[0, 3, 4, 5, 6, 7], :], df_of_set_wpid_draft1[df_of_set_wpid_draft1['key_mlbam']==544725]], axis=0)
df_of_set_wpid_draft2

,rank,player,team,elig_pos,pos_rank,age,name_last_lookup,name_first_lookup,name_last,key_mlbam,Team,WAR
0,1,Mike Trout,LAA,OF,OF1,28,trout,mike,trout,545361,Angels,9.8
3,4,Mookie Betts,BOS,OF,OF4,26,betts,mookie,betts,605141,Red Sox,10.4
4,6,Cody Bellinger,LAD,1B/OF,1B1,24,bellinger,cody,bellinger,641355,Dodgers,3.6
5,282,Jesse Winker,CIN,OF,OF90,26,winker,jesse,winker,608385,Reds,0.9
6,286,Ian Desmond,COL,1B/OF,1B38,33,desmond,ian,desmond,435622,Rockies,-0.6
7,297,Anthony Santander,BAL,OF,OF92,24,santander,anthony,santander,623993,Orioles,-0.4
11,300,Leury Garcia,CWS,OF/SS,OF94,28,garcia,leury,garcia,544725,White Sox,0.1


In [49]:
from itertools import combinations

In [61]:
df_pairs = pd.DataFrame()
df_pairs['pairs'] = [pair for pair in combinations(df_of_set_wpid_draft2['name_last'], 2)]
df_pairs['pairs_pid'] = [pair for pair in combinations(df_of_set_wpid_draft2['key_mlbam'], 2)]
df_pairs['WAR_avg'] = df_of_set_wpid_draft2['key_mlbam']

,pairs,pairs_pid
0,"(trout, betts)","(545361, 605141)"
1,"(trout, bellinger)","(545361, 641355)"
2,"(trout, winker)","(545361, 608385)"
3,"(trout, desmond)","(545361, 435622)"
4,"(trout, santander)","(545361, 623993)"
5,"(trout, garcia)","(545361, 544725)"
6,"(betts, bellinger)","(605141, 641355)"
7,"(betts, winker)","(605141, 608385)"
8,"(betts, desmond)","(605141, 435622)"
9,"(betts, santander)","(605141, 623993)"


In [83]:
for i, pid_pair in enumerate(df_pairs['pairs_pid']):
    war1 = (df_of_set_wpid_draft2.loc[df_of_set_wpid_draft2['key_mlbam']==pid_pair[0], 'WAR']).iloc[0]
    war2 = (df_of_set_wpid_draft2.loc[df_of_set_wpid_draft2['key_mlbam']==pid_pair[1], 'WAR']).iloc[0]
    df_pairs.loc[i, 'WAR_avg'] = np.mean([war1, war2])
    df_pairs.loc[i, 'WAR_diff'] = np.abs(war1-war2)

In [84]:
df_pairs

,pairs,pairs_pid,WAR_avg,WAR_diff
0,"(trout, betts)","(545361, 605141)",10.10,0.6
1,"(trout, bellinger)","(545361, 641355)",6.70,6.2
2,"(trout, winker)","(545361, 608385)",5.35,8.9
3,"(trout, desmond)","(545361, 435622)",4.60,10.4
4,"(trout, santander)","(545361, 623993)",4.70,10.2
5,"(trout, garcia)","(545361, 544725)",4.95,9.7
6,"(betts, bellinger)","(605141, 641355)",7.00,6.8
7,"(betts, winker)","(605141, 608385)",5.65,9.5
8,"(betts, desmond)","(605141, 435622)",4.90,11.0
9,"(betts, santander)","(605141, 623993)",5.00,10.8


# Add dates from statcast

Where players are playing on same days

In [ ]:
# Verifying query
#  game_date      | batter 
# ---------------------+--------
#  2019-09-02 00:00:00 | 435622  x
#  2019-09-03 00:00:00 | 435622  x
#  2019-09-04 00:00:00 | 435622
#  2019-09-07 00:00:00 | 435622  x
#  2019-09-08 00:00:00 | 435622  x
#  2019-09-11 00:00:00 | 435622  x
#  2019-09-12 00:00:00 | 435622  x
#  2019-09-13 00:00:00 | 435622
#  2019-09-14 00:00:00 | 435622  x
#  2019-09-16 00:00:00 | 435622  x
#  2019-09-18 00:00:00 | 435622  x
#  2019-09-20 00:00:00 | 435622  x
#  2019-09-21 00:00:00 | 435622  x
#  2019-09-22 00:00:00 | 435622
#  2019-09-24 00:00:00 | 435622
#  2019-09-25 00:00:00 | 435622
#  2019-09-26 00:00:00 | 435622
#  2019-09-28 00:00:00 | 435622
#  2019-09-29 00:00:00 | 435622

In [ ]:
# SQL query to get get common dates ( this was verified manually)
# I'll assume they started that game

# SELECT * FROM
# (SELECT DISTINCT statcast.game_date, statcast.batter FROM statcast 
# WHERE statcast.batter= 435622 AND 
# statcast.game_date>'2019-09-01') p1
# JOIN 
# (SELECT DISTINCT statcast.game_date, statcast.batter FROM statcast 
# WHERE statcast.batter= 623993 AND 
# statcast.game_date>'2019-09-01') p2
# ON p1.game_date=p2.game_date;

In [134]:
def get_common_playing_dates(player_id1, player_id2, starting_date):

    # Test set
    # player_id1 = 435622
    # player_id2 = 623993
    # starting_date = "2019-09-01"

    sql_query = (
        """
        SELECT * FROM 
        (SELECT DISTINCT statcast.game_date, statcast.batter 
        FROM statcast 
        WHERE statcast.batter= 
        """
        + str(player_id1)
        + """
        AND statcast.game_date>
        """
        + str(starting_date)
        + ")"
        + """ 
        p1 
        JOIN 
        (SELECT DISTINCT statcast.game_date, statcast.batter 
        FROM statcast 
        WHERE statcast.batter= 
        """
        + str(player_id2)
        + """
        AND statcast.game_date>
        """
        + str(starting_date)
        + ")"
        + """ 
        p2
        ON p1.game_date=p2.game_date;
        """
    )

    print(sql_query)

    df_common_dates = pd.read_sql_query(sql_query, con)
    return df_common_dates

<IPython.core.display.Javascript object>

In [203]:
# Trout and Garcia (545361, 544725)
# pid1 = 545361
# pid2 = 544725

# Trout and Santander (545361, 623993) - Trout got injured!
# pid1 = 545361
# pid2 = 623993

# Betts and Garcia (605141, 544725)
# pid1 = 605141
# pid2 = 544725
 

# Reynolds and Garcia (668804, 544725)
# Bryan Reynolds (PIT)

# pid1 = 668804
# pid2 = 544725


# Santander and Garcia (623993, 544725)

pid1 = 623993
pid2 = 544725

date_cutoff = "'2019-08-01'"  # note internal quotes necessary for queyr

df_overlap_p1p2_dates = get_common_playing_dates(pid1, pid2, date_cutoff)
df_overlap_p1p2_dates.columns = ['game_date_p1', 'pid_p1', 'game_date_p2', 'pid_p2']



        SELECT * FROM 
        (SELECT DISTINCT statcast.game_date, statcast.batter 
        FROM statcast 
        WHERE statcast.batter= 
        623993
        AND statcast.game_date>
        '2019-08-01') 
        p1 
        JOIN 
        (SELECT DISTINCT statcast.game_date, statcast.batter 
        FROM statcast 
        WHERE statcast.batter= 
        544725
        AND statcast.game_date>
        '2019-08-01') 
        p2
        ON p1.game_date=p2.game_date;
        


<IPython.core.display.Javascript object>

In [204]:
df_overlap_p1p2_dates.shape

(31, 4)

<IPython.core.display.Javascript object>

In [205]:
# Format date time
[i.strftime("%Y-%m-%d") for i in df_overlap_p1p2_dates["game_date_p1"]]

['2019-08-02',
 '2019-08-03',
 '2019-08-04',
 '2019-08-05',
 '2019-08-06',
 '2019-08-07',
 '2019-08-09',
 '2019-08-10',
 '2019-08-16',
 '2019-08-17',
 '2019-08-18',
 '2019-08-19',
 '2019-08-20',
 '2019-08-21',
 '2019-08-22',
 '2019-08-23',
 '2019-08-24',
 '2019-08-25',
 '2019-08-27',
 '2019-08-28',
 '2019-08-30',
 '2019-08-31',
 '2019-09-01',
 '2019-09-02',
 '2019-09-03',
 '2019-09-10',
 '2019-09-11',
 '2019-09-12',
 '2019-09-16',
 '2019-09-18',
 '2019-09-21']

<IPython.core.display.Javascript object>